In [1]:
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("seif", "rafting123456"),
    ca_certs=r"C:\Users\dell\Desktop\elasticsearch-8.12.2\config\certs\http_ca.crt"
)
es.ping()

True

## Prepare the data

In [15]:
import pandas as pd

df = pd.read_csv("./dataset/D-b-nhams-Womens-Wdding-Coll-ctions-Data.csv").loc[:2000]

df.head()

,product_url,product_name,brand,sku,image_url,star_rating,number_of_reviews,MRP,sale_price,discount_percentage,colour,description,details_and_care,style_1,category,material
0,https://www.debenhams.com/product/jon-richard-...,Gold Bracelet Extender,Jon Richard,SKU: M5051024711187,https://media.boohoo.com/i/boohoo/m50510247111...,5,1,1.5,1.20,20,Gold,"Love your bracelet, but want to wear it an inc...",Fastening: Fold Over Clasp | Dimensions: 25mm ...,Bracelet,Accessories,Not Available
1,https://www.debenhams.com/product/jon-richard-...,Gold Extension Chain,Jon Richard,SKU: M5051024283509,https://media.boohoo.com/i/boohoo/m50510242835...,5,1,1.5,1.20,20,Gold,"Love your bracelet, but want to wear it an inc...",Fastening: Fold Over Clasp Bracelet Extender |...,Bracelet,Accessories,Not Available
2,https://www.debenhams.com/product/jon-richard-...,Silver Plated Extension Chain,Jon Richard,SKU: M5051024711170,https://media.boohoo.com/i/boohoo/m50510247111...,5,2,1.5,1.20,20,Silver,Jon Richard Silver Plated Extension Chain.\nSt...,Fold Over Clasp | 30mm | 50mm,Bracelet,Accessories,Not Available
3,https://www.debenhams.com/product/jon-richard-...,Two Tone Bullet Back Pack of 4,Jon Richard,SKU: M5051024283530,https://media.boohoo.com/i/boohoo/m50510242835...,5,4,2.0,1.60,20,Multi,Lost the backs to your favourite pair of earri...,Fastening: Post and Bullet Back | Dimensions: ...,Earrings,Accessories,Not Available
4,https://www.debenhams.com/product/simply-silve...,Pouch Gift Box,Simply Silver,SKU: M5051024636206,https://media.boohoo.com/i/boohoo/m50510246362...,5,1,3.0,1.68,44,Dark Brown,Whether you are gifting a loved one or self-gi...,Fastening: ribbon fastening | Dimensions: 70mm...,Jewellery Box,Accessories,Not Available


In [16]:
df.isna().value_counts()

product_url  product_name  brand  sku    image_url  star_rating  number_of_reviews  MRP    sale_price  discount_percentage  colour  description  details_and_care  style_1  category  material
False        False         False  False  False      False        False              False  False       False                False   False        False             False    False     False       998
Name: count, dtype: int64

In [17]:
# na values generate error in elastic search
df.fillna("None")

,product_url,product_name,brand,sku,image_url,star_rating,number_of_reviews,MRP,sale_price,discount_percentage,colour,description,details_and_care,style_1,category,material
0,https://www.debenhams.com/product/jon-richard-...,Gold Bracelet Extender,Jon Richard,SKU: M5051024711187,https://media.boohoo.com/i/boohoo/m50510247111...,5,1,1.5,1.20,20,Gold,"Love your bracelet, but want to wear it an inc...",Fastening: Fold Over Clasp | Dimensions: 25mm ...,Bracelet,Accessories,Not Available
1,https://www.debenhams.com/product/jon-richard-...,Gold Extension Chain,Jon Richard,SKU: M5051024283509,https://media.boohoo.com/i/boohoo/m50510242835...,5,1,1.5,1.20,20,Gold,"Love your bracelet, but want to wear it an inc...",Fastening: Fold Over Clasp Bracelet Extender |...,Bracelet,Accessories,Not Available
2,https://www.debenhams.com/product/jon-richard-...,Silver Plated Extension Chain,Jon Richard,SKU: M5051024711170,https://media.boohoo.com/i/boohoo/m50510247111...,5,2,1.5,1.20,20,Silver,Jon Richard Silver Plated Extension Chain.\nSt...,Fold Over Clasp | 30mm | 50mm,Bracelet,Accessories,Not Available
3,https://www.debenhams.com/product/jon-richard-...,Two Tone Bullet Back Pack of 4,Jon Richard,SKU: M5051024283530,https://media.boohoo.com/i/boohoo/m50510242835...,5,4,2.0,1.60,20,Multi,Lost the backs to your favourite pair of earri...,Fastening: Post and Bullet Back | Dimensions: ...,Earrings,Accessories,Not Available
4,https://www.debenhams.com/product/simply-silve...,Pouch Gift Box,Simply Silver,SKU: M5051024636206,https://media.boohoo.com/i/boohoo/m50510246362...,5,1,3.0,1.68,44,Dark Brown,Whether you are gifting a loved one or self-gi...,Fastening: ribbon fastening | Dimensions: 70mm...,Jewellery Box,Accessories,Not Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,https://www.debenhams.com/product/monsoon-lori...,'Lori' Lace Maxi Dress,Monsoon,SKU: M5045462870964,https://media.boohoo.com/i/boohoo/m50454628709...,Not Available,Not Available,130.0,65.00,50,Blue,Lovely Lori is perfect for summer special occa...,"Outer, 100% Polyester. Lining, 100% Polyester....",Maxi Dress,Dresses,Polyester
994,https://www.debenhams.com/product/monsoon-adri...,'Adrianna' Embellished Maxi Dress,Monsoon,SKU: M5045462871923,https://media.boohoo.com/i/boohoo/m50454628719...,Not Available,Not Available,150.0,75.00,50,Blue,Achieve a warm reception always when styling t...,"Outer, 100% Recycled Polyester. Lining, 100% R...",Maxi Dress,Dresses,Polyester
995,https://www.debenhams.com/product/goddiva-chif...,Chiffon Off The Shoulder Maxi With Wings,Goddiva,SKU: M5056120200320,https://media.boohoo.com/i/boohoo/m50561202003...,Not Available,Not Available,94.0,75.20,20,Dark Pink,Our stunning Chiffon Off The Shoulder Maxi wit...,"Knitted, 100% Polyester, Do not dry clean cold...",Maxi Dress,Dresses,Polyester
996,https://www.debenhams.com/product/monsoon-anna...,'Anna' Embroidered Broderie Maxi Dress,Monsoon,SKU: M5045463017450,https://media.boohoo.com/i/boohoo/m50454630174...,5,1,170.0,85.00,50,Ivory,A romantic piece for summer occasions. Anna fe...,"Outer, 100% Cotton. Lining, 100% Cotton. L: 13...",Maxi Dress,Dresses,Cotton


## Convert relevant field ro vector using BERT model

In [18]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

In [20]:
df["DescriptionVector"] = df["description"].apply(lambda x: model.encode(x))

In [21]:

df.head()

,product_url,product_name,brand,sku,image_url,star_rating,number_of_reviews,MRP,sale_price,discount_percentage,colour,description,details_and_care,style_1,category,material,DescriptionVector
0,https://www.debenhams.com/product/jon-richard-...,Gold Bracelet Extender,Jon Richard,SKU: M5051024711187,https://media.boohoo.com/i/boohoo/m50510247111...,5,1,1.5,1.20,20,Gold,"Love your bracelet, but want to wear it an inc...",Fastening: Fold Over Clasp | Dimensions: 25mm ...,Bracelet,Accessories,Not Available,"[0.0399403, -0.012686805, -0.0145875625, 0.012..."
1,https://www.debenhams.com/product/jon-richard-...,Gold Extension Chain,Jon Richard,SKU: M5051024283509,https://media.boohoo.com/i/boohoo/m50510242835...,5,1,1.5,1.20,20,Gold,"Love your bracelet, but want to wear it an inc...",Fastening: Fold Over Clasp Bracelet Extender |...,Bracelet,Accessories,Not Available,"[0.0399403, -0.012686805, -0.0145875625, 0.012..."
2,https://www.debenhams.com/product/jon-richard-...,Silver Plated Extension Chain,Jon Richard,SKU: M5051024711170,https://media.boohoo.com/i/boohoo/m50510247111...,5,2,1.5,1.20,20,Silver,Jon Richard Silver Plated Extension Chain.\nSt...,Fold Over Clasp | 30mm | 50mm,Bracelet,Accessories,Not Available,"[0.072946034, -0.028632248, 0.010856216, 0.029..."
3,https://www.debenhams.com/product/jon-richard-...,Two Tone Bullet Back Pack of 4,Jon Richard,SKU: M5051024283530,https://media.boohoo.com/i/boohoo/m50510242835...,5,4,2.0,1.60,20,Multi,Lost the backs to your favourite pair of earri...,Fastening: Post and Bullet Back | Dimensions: ...,Earrings,Accessories,Not Available,"[-0.023067847, -0.04054853, 0.0088367835, 0.04..."
4,https://www.debenhams.com/product/simply-silve...,Pouch Gift Box,Simply Silver,SKU: M5051024636206,https://media.boohoo.com/i/boohoo/m50510246362...,5,1,3.0,1.68,44,Dark Brown,Whether you are gifting a loved one or self-gi...,Fastening: ribbon fastening | Dimensions: 70mm...,Jewellery Box,Accessories,Not Available,"[0.02545201, -0.017068028, -0.0038147573, 0.03..."


## Create new index in ElasticSearch!


In [23]:
# from indexMapping import indexMapping
from indexMappingJewelery import indexMapping


es.indices.create(index="all_products_jewlery", mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'all_products_jewlery'})

## Ingest the data into index


In [24]:
# transforming the data into an array
record_list = df.to_dict("records")
record_list[0]

{'product_url': 'https://www.debenhams.com/product/jon-richard-gold-bracelet-extender_p-4b287644-721e-4873-9b1c-0917634f010c?colour=Gold',
 'product_name': 'Gold Bracelet Extender',
 'brand': 'Jon Richard',
 'sku': 'SKU: M5051024711187',
 'image_url': 'https://media.boohoo.com/i/boohoo/m5051024711187_gold_xl.jpeg?sm=c&fmt=auto&qlt=75&w=%7B400%2A%24dpr%7D&dpr=1',
 'star_rating': '5',
 'number_of_reviews': '1',
 'MRP': 1.5,
 'sale_price': 1.2,
 'discount_percentage': '20',
 'colour': 'Gold',
 'description': 'Love your bracelet, but want to wear it an inch longer? Our Jon Richard pack of two extension chains will give you the flexibility to wear it at different lengths. A must-have accessory to switch up your look.\nStyle: Bracelet',
 'details_and_care': 'Fastening: Fold Over Clasp | Dimensions: 25mm | Drop Dimensions: 10mm',
 'style_1': 'Bracelet',
 'category': 'Accessories',
 'material': 'Not Available',
 'DescriptionVector': array([ 3.99403013e-02, -1.26868049e-02, -1.45875625e-02,  1.

In [26]:
for record in record_list:
    try:
        es.index(index="all_products_jewlery", document=record, id=record["product_url"])
    except Exception as e:
        print(e)

In [28]:
es.count(index="all_products_jewlery")

ObjectApiResponse({'count': 998, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Search the data

In [32]:
input_keyword = "product contains an flowers"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "DescriptionVector",
    "query_vector" : vector_of_input_keyword,
    "k" : 2,
    "num_candidates" : 1799, 
}

res = es.knn_search(index="all_products_jewlery", knn=query , source=["product_name","Description"])
res["hits"]["hits"]

C:\Users\dell\AppData\Local\Temp\ipykernel_22000\2640262123.py:11: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="all_products_jewlery", knn=query , source=["product_name","Description"])


[{'_index': 'all_products_jewlery',
  '_id': 'https://www.debenhams.com/product/jon-richard-gold-plated-leaf-hair-vine_p-feb9166a-2d27-4ec5-b288-9f70097e7323?colour=Gold',
  '_score': 0.46991265,
  '_source': {'product_name': 'Gold Plated Leaf Hair Vine'}},
 {'_index': 'all_products_jewlery',
  '_id': 'https://www.debenhams.com/product/jon-richard-square-gift-box_p-3bec18f2-2cf0-4f0d-ae1f-4d39e2854e1f?colour=Pink',
  '_score': 0.44979295,
  '_source': {'product_name': 'Square Gift Box'}}]